**Author:** Arefeh Sherafati

**Data loading author:** steeve.laquitaine@epfl.ch: Courtesy of Neuromatch Academy Computational Neuroscience Track.

## Paper Summary: *A Switching Bayesian Observer Model for Human Perceptual Estimation*  
**Paper**: [Laquitaine & Gardner, Neuron, 2018](https://doi.org/10.1016/j.neuron.2017.12.011)

---

### Main Objective
To understand how **prior expectations** and **sensory evidence** interact in human perception, especially during motion direction estimation under uncertainty.

---

### Experimental Design
- **Subjects**: 12 human participants  
- **Task**: Motion direction estimation  
- **Manipulations**:
  - **Sensory uncertainty**: Controlled via motion coherence (low coherence = noisier input)
  - **Priors**: Directions drawn from different underlying distributions (some more frequent)

- **Data**: Continuous direction estimates across ~83,000 trials

---

### Key Questions
- Do humans combine prior knowledge and sensory input in a **Bayesian-optimal** way?  
- Or is behavior better explained by **heuristics**, such as switching between prior and sensory evidence?

---

### Modeling Approach
Compared three observer models:
- **Bayesian model**: Integrates prior + likelihood into a posterior
- **Switching model**: Chooses **either** prior **or** likelihood on each trial
- **Weighted mixture model**: Combines both with flexible weighting

Fitted models to:
- Trial-level behavior (errors and reaction times)  
- Used **likelihood-based model comparison** to evaluate fits

---

### Key Findings
- Human observers **do not always perform Bayesian integration**.
- Behavior is better explained by a **switching observer model**, suggesting:
  - People flexibly rely on **prior** or **sensory** information depending on context or confidence
- This **switching strategy** yields **more robust behavior** under uncertainty.

---

### Broader Implications
- Challenges the notion that perception is always **Bayesian**  
- Suggests perceptual inference may be **context-dependent**  
- Introduces a **novel modeling framework** (switching observer) that better reflects human behavior

---

### Key Contributions
- Introduction of a **switching observer model**  
- Collection of a large-scale behavioral dataset (~83k trials)  
- Empirical demonstration that switching strategy outperforms strict Bayesian integration

In [ ]:
# @title Dependencies
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib import rcParams
import numpy as np
import os, requests
from scipy.stats import norm, uniform
from matplotlib.colors import ListedColormap
from numpy import pi
from copy import copy
from matplotlib import colors

In [ ]:
# @title Figure settings
rcParams['figure.figsize'] = [20, 4]
rcParams['font.size'] = 11
rcParams['axes.spines.top'] = False
rcParams['axes.spines.right'] = False
rcParams['figure.autolayout'] = True

In [ ]:
# @title Data retrieval

url = "https://github.com/steevelaquitaine/projInference/raw/gh-pages/data/csv/data01_direction4priors.csv"
try:
  RequestAPI = requests.get(url)
except requests.ConnectionError:
  print("Failed to download data. Please contact steeve.laquitaine@epfl.ch")
else:
  if RequestAPI.status_code != requests.codes.ok:
    print("Failed to download data. Please contact steeve.laquitaine@epfl.ch")
  else:
    with open("data01_direction4priors.csv", "wb") as fid:
      fid.write(RequestAPI.content)

In [ ]:
# @title Data loading
data = pd.read_csv("data01_direction4priors.csv")
data.head()

### Data dictionary

`data` contains sessions from 12 human subjects, data from [Laquitaine & Gardner, 2018](https://doi.org/10.1016/j.neuron.2017.12.011).

Subjects had to estimate the direction of stimulus motion directions.

* `data['trial_index']`: trial index
* `data['trial_time']`: time at which trial starts with th e central fixation dot
* `data['response_arrow_start_angle']`: the angle of the response arrow at the start of the response phase.
* `data['motion_direction']`: the stimulus motion direction
* `data['motion_coherence']`: the stimulus motion coherence
* `data['estimate_x']`: x cartesian coordinate of the stimulus motion direction
* `data['estimate_y']`: y cartesian coordinate of the stimulus motion direction
* `data['reaction_time']`: subject's reaction time
* `data['raw_response_time']`: subject response time since the start of the run (of about 200 trials)
* `data['prior_std']`: It is the standard deviation of the statistical distribution (motion direction generative process over trials, which we call "experimental prior") from which we sampled the stimulus motion direction displayed in each trial.
* `data['prior_mean']`: the most frequently displayed motion direction. It is the mean of the statistical distribution (motion direction generative process over trials, which we call "experimental prior") from which we sampled the stimulus motion direction displayed in each trial.
* `data['subject_id']`: the id of the subject for which behavior was recorded.
* `data['experiment_name']`: the name of the experiment. This dataaset only contains the "data01_direction4priors" experiment in which subject underwent a task in which four motion direction were sampled from one of four priors with 10, 40, 60 and 80 degree standard deviations in each block of about 200 trials. The mean of the "experimental prior"  was fixed at 225 deg.
* `data['experiment_id']`: the id of the experiment.
* `data['session_id']`: the id of the session.
* `data['run_id']`: the id of the run.


The complete original dataset is stored in .mat files here: https://data.mendeley.com/datasets/nxkvtrj9ps/1.

In [ ]:
# @title Utils
def build_generative_model(std, mean):
    """create hidden generative process for motion direction
    the experimental Gaussian prior
    """
    # set direction state space
    x = np.arange(1, 360, 1)

    # calculate probability distribution
    pdf = norm.pdf(x, loc=mean, scale=std)
    pdf /= sum(pdf)
    return pdf

def generate_directions(mean, std, n_trials, seed):
    return np.round(norm.rvs(loc=mean, scale=std, size=n_trials, random_state=seed))


def learn_generative_process(motion_directions, learning_rate, x, n_trials):
    """learn the generative process
    """
    # set subject initial belief state
    initial_prior = uniform.pdf(x, loc=x[0], scale=x[-1])
    initial_prior /= sum(initial_prior)
    prior = copy.copy(initial_prior)
    observed = np.zeros((len(x)))
    prediction_errors = []
    priors = []

    for old_trial in range(0, n_trials):

        # locate the observed state component
        state_loc = np.where(x == int(motion_directions[old_trial]))[0][0]

        # compute its state prediction error
        state_pred_error = learning_rate * (1 - prior[state_loc])

        # use the prediction error to update the state belief
        prior[state_loc] = prior[state_loc] + state_pred_error
        prior /= sum(prior)

        # tape
        prediction_errors.append(state_pred_error)
        priors.append(copy.copy(prior))
    return priors

# circular statistics utils
# -------------------
def get_cartesian_to_deg(
    x: np.ndarray, y: np.ndarray, signed: bool
) -> np.ndarray:
    """convert cartesian coordinates to
    angles in degree
    Args:
        x (np.ndarray): x coordinate
        y (np.ndarray): y coordinate
        signed (boolean): True (signed) or False (unsigned)
    Usage:
        .. code-block:: python
            import numpy as np
            from bsfit.nodes.cirpy.utils import get_cartesian_to_deg
            x = np.array([1, 0, -1, 0])
            y = np.array([0, 1, 0, -1])
            degree = get_cartesian_to_deg(x,y,False)
            # Out: array([  0.,  90., 180., 270.])
    Returns:
        np.ndarray: angles in degree
    """
    # convert to radian (ignoring divide by 0 warning)
    with np.errstate(divide="ignore"):
        degree = np.arctan(y / x)

    # convert to degree and adjust based
    # on quadrant
    for ix in range(len(x)):
        if (x[ix] >= 0) and (y[ix] >= 0):
            degree[ix] = degree[ix] * 180 / np.pi
        elif (x[ix] == 0) and (y[ix] == 0):
            degree[ix] = 0
        elif x[ix] < 0:
            degree[ix] = degree[ix] * 180 / np.pi + 180
        elif (x[ix] >= 0) and (y[ix] < 0):
            degree[ix] = degree[ix] * 180 / np.pi + 360

    # if needed, convert signed to unsigned
    if not signed:
        degree[degree < 0] = degree[degree < 0] + 360
    return degree

def get_deg_to_rad(deg: np.array, signed: bool):
    """convert angles in degree to radian
    Args:
        deg (np.array): angles in degree
        signed (bool): True (signed) or False (unsigned)
    Usage:
        .. code-block:: python
            import numpy as np
            from bsfit.nodes.cirpy.utils import get_deg_to_rad
            radians = get_deg_to_rad(np.array([0, 90, 180, 270], True)
            Out: array([ 0., 1.57079633, 3.14159265, -1.57079633])
    Returns:
        np.ndarray: angles in radian
    """
    # get unsigned radians (1:2*pi)
    rad = (deg / 360) * 2 * pi

    # get signed radians(-pi:pi)
    if signed:
        rad[deg > 180] = (deg[deg > 180] - 360) * (
            2 * pi / 360
        )
    return rad

def get_polar_to_cartesian(
    angle: np.ndarray, radius: float, type: str
) -> dict:
    """convert angle in degree or radian to cartesian coordinates
    Args:
        angle (np.ndarray): angles in degree or radian
        radius (float): radius
        type (str): "polar" or "radian"
    Usage:
        .. code-block:: python
            import numpy as np
            from bsfit.nodes.cirpy.utils import get_polar_to_cartesian
            degree = np.array([0, 90, 180, 270])
            cartesian = get_polar_to_cartesian(degree, 1, "polar")
            cartesian.keys()

            # Out: dict_keys(['deg', 'rad', 'cart'])

            cartesian["cart"]

            # Out: array([[ 1.,  0.],
            #            [ 0.,  1.],
            #            [-1.,  0.],
            #            [-0., -1.]])
    Returns:
        dict: _description_
    """
    # convert to radian if needed
    theta = dict()
    if type == "polar":
        theta["deg"] = angle
        theta["rad"] = angle * np.pi / 180
    elif type == "radian":
        theta["deg"] = get_deg_to_rad(angle, False)
        theta["rad"] = angle

    # convert to cartesian coordinates
    x = radius * np.cos(theta["rad"])
    y = radius * np.sin(theta["rad"])

    # round to 10e-4
    x = np.round(x, 4)
    y = np.round(y, 4)

    # reshape as (N angles x 2 coord)
    theta["cart"] = np.vstack([x, y]).T
    return theta

def get_circ_weighted_mean_std(
    angle: np.ndarray, proba: np.ndarray, type: str
) -> dict:
    """calculate circular data statistics
    Args:
        angle (np.ndarray): angles in degree or cartesian coordinates
        proba (np.ndarray): each angle's probability of occurrence
        type (str): "polar" or "cartesian"
    Usage:
        .. code-block:: python
            import numpy as np
            from bsfit.nodes.cirpy.utils import get_circ_weighted_mean_std
            degree = np.array([358, 0, 2, 88, 90, 92])
            proba = np.array([1, 1, 1, 1, 1, 1])/6
            output = get_circ_weighted_mean_std(degree, proba, "polar")
            output.keys()
            # Out: dict_keys(['coord_all', 'deg_all', 'coord_mean', 'deg_mean',
            #               'deg_all_for_std', 'deg_mean_for_std', 'deg_var',
            #               'deg_std', 'deg_sem'])
            output["deg_mean"]
            # Out: array([45.])
            output["deg_std"]
            # array([45.02961988])
    Returns:
        (dict): angle summary statistics (mean, std, var, sem)

    Raises:
        ValueError: type is not "polar" or "cartesian"
    """

    angle = angle.copy()

    # if polar, convert to cartesian
    if type == "polar":
        radius = 1
        coord = get_polar_to_cartesian(
            angle, radius=radius, type="polar"
        )
    elif type == "cartesian":
        coord = angle
    else:
        raise ValueError(
            """ "type" can either be "polar" or "cartesian" value """
        )

    # store angles
    data = dict()
    data["coord_all"] = coord["cart"]
    data["deg_all"] = coord["deg"]

    # calculate mean
    # ..............
    proba_for_mean = np.tile(proba[:, None], 2)
    data["coord_mean"] = np.sum(
        proba_for_mean * data["coord_all"], 0
    )
    data["coord_mean"] = data["coord_mean"][:, None]
    data["deg_mean"] = get_cartesian_to_deg(
        data["coord_mean"][0],
        data["coord_mean"][1],
        signed=False,
    )

    # calculate std
    # ..............
    n_data = len(data["deg_all"])
    data["deg_all_for_std"] = data["deg_all"]
    data["deg_mean_for_std"] = np.tile(
        data["deg_mean"], n_data
    )

    # apply corrections
    # when 0 <= mean <= 180
    if data["deg_mean"] + 180 <= 360:
        for ix in range(n_data):
            if (
                data["deg_all"][ix]
                >= data["deg_mean"] + 180
            ):
                data["deg_all_for_std"][ix] = (
                    data["deg_all"][ix] - 360
                )
    else:
        # when 180 <= mean <= 360
        for ix in range(n_data):
            if (
                data["deg_all"][ix]
                <= data["deg_mean"] - 180
            ):
                data["deg_mean_for_std"][ix] = (
                    data["deg_mean"] - 360
                )

    # calculate variance, standard deviation and
    # standard error to the mean
    data["deg_var"] = np.array(
        [
            sum(
                proba
                * (
                    data["deg_all_for_std"]
                    - data["deg_mean_for_std"]
                )
                ** 2
            )
        ]
    )
    data["deg_std"] = np.sqrt(data["deg_var"])
    data["deg_sem"] = data["deg_std"] / np.sqrt(n_data)
    return data

def get_signed_angle(
    origin: np.ndarray, destination: np.ndarray, type: str
):
    """get the signed angle difference between origin and destination angles
    Args:
        origin (np.ndarray): origin angle
        destination (np.ndarray): destination angle
        type (str): angle type ("polar", "radian", "cartesian")
    Usage:
        .. code-block:: python
            angle = get_signed_angle(90, 45, 'polar')

            # Out: array([45.])

            angle = get_signed_angle(90, 45, 'radian')
            # Out: array([58.3103779])
            origin = np.array([[0, 1]])
            destination = np.array([[1, 0]])
            angle = get_signed_angle(origin, destination, "cartesian")

            # Out: array([90.])
    Returns:
        (np.ndarray): signed angle differences
    """

    # convert to cartesian coordinates
    if type == "polar" or type == "radian":
        origin_dict = get_polar_to_cartesian(
            origin, radius=1, type=type
        )
        destination_dict = get_polar_to_cartesian(
            destination, radius=1, type=type
        )
    elif type == "cartesian":
        origin_dict = dict()
        destination_dict = dict()
        origin_dict["cart"] = origin
        destination_dict["cart"] = destination

    # get coordinates
    xV1 = origin_dict["cart"][:, 0]
    yV1 = origin_dict["cart"][:, 1]
    xV2 = destination_dict["cart"][:, 0]
    yV2 = destination_dict["cart"][:, 1]

    # Calculate the angle separating the
    # two vectors in degrees
    angle = -(180 / np.pi) * np.arctan2(
        xV1 * yV2 - yV1 * xV2, xV1 * xV2 + yV1 * yV2
    )
    return angle

def get_combination_set(database: np.ndarray):
    """get the set of row combinations

    Args:
        database (np.ndarray): an N-D array

    Returns:
        (np.ndarray, np.ndarray, np.ndarray): `combs` is the set
        of combinations, `b` are the row indices for each combination
        in database, `c` are the rows indices for each combination in
        combs.
    """
    combs, ia, ic = np.unique(
        database,
        return_index=True,
        return_inverse=True,
        axis=0,
    )
    return (combs, ia, ic)

def get_data_stats(data: pd.Series, output: dict):
    """calculate data statistics

    Args:
        data (pd.Series): stimulus feature estimates
        output (dict): ::

            'PestimateGivenModel': estimate probabilities
            'map': max-a-posteriori percepts
            'conditions': task conditions

    Returns:
        (dict): returns data mean and std to output
    """
    # get conditions
    cond = output["conditions"]

    # initialise statistics
    data_mean = []
    data_std = []

    # get set of conditions
    cond_set, ix, _ = get_combination_set(cond)

    # record stats by condition
    for c_i in range(len(cond_set)):

        # find condition's instances
        loc_1 = cond[:, 0] == cond_set[c_i, 0]
        loc_2 = cond[:, 1] == cond_set[c_i, 1]
        loc_3 = cond[:, 2] == cond_set[c_i, 2]

        # get associated data
        data_c_i = data.values[loc_1 & loc_2 & loc_3]

        # set each instance with equal probability
        trial_proba = np.tile(
            1 / len(data_c_i), len(data_c_i)
        )

        # get statistics
        stats = get_circ_weighted_mean_std(
            data_c_i, trial_proba, type="polar",
        )

        # record statistics
        data_mean.append(stats["deg_mean"])
        data_std.append(stats["deg_std"])

    # record statistics
    output["data_mean"] = np.array(data_mean)
    output["data_std"] = np.array(data_std)

    # record their condition
    output["conditions"] = cond_set
    return output

## Hypothesis I: A Basic Linear Regression Model of Circular Distance Prediction

**Goal**  
Calculate the circular distances between past trial estimates and current trial displayed directions.  
- Split the data into train and test sets.  
- Train a linear regression to predict the error from that feature.  
- Report R² on the test set.  

---

### Assumptions

- **Circular distances** are computed modulo 360 (wrapped angular differences).
- The **prediction target** is the **angular error**, defined as:

$$
\text{angular error} = \text{estimate} - \text{stimulus direction}
$$

  (wrapped within [–180°, +180°] or [0°, 360°], depending on convention)

- The main **feature** is the **circular distance between the previous trial's estimate and the current trial’s stimulus direction**.


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

# --- Step 1: Prepare Data ---
df = data.copy()

# Use existing function from notebook to convert (x, y) to degrees
df["estimate_deg"] = get_cartesian_to_deg(df["estimate_x"].values, df["estimate_y"].values, signed=False)

# Sort to maintain temporal trial order within each subject
df = df.sort_values(["subject_id", "session_id", "run_id", "trial_index"])

# Create feature: circular distance between previous estimate and current stimulus
df["prev_estimate"] = df.groupby("subject_id")["estimate_deg"].shift(1)

# Use built-in helper to compute circular distances
def circular_distance_deg(a, b):
    return np.angle(np.exp(1j * np.deg2rad(a - b)), deg=True)

df["circ_dist_prev_est_to_stim"] = circular_distance_deg(df["motion_direction"], df["prev_estimate"])

# Create target: circular error = estimate - stimulus direction (wrapped)
df["circular_error"] = circular_distance_deg(df["estimate_deg"], df["motion_direction"])

# Drop NaNs
df = df.dropna(subset=["circ_dist_prev_est_to_stim", "circular_error"])

# --- Step 2: Split into train/test ---
X = df[["circ_dist_prev_est_to_stim"]].values
y = df["circular_error"].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# --- Step 3: Train linear regression ---
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)

# --- Step 4: Report R² score ---
r2 = r2_score(y_test, y_pred)
print(f"R² score on test set: {r2:.3f}")


### Scatter Plot with Line of Best Fit

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8,5))
plt.scatter(X_test, y_test, alpha=0.3, label="True")
plt.scatter(X_test, y_pred, color='red', alpha=0.3, label="Predicted")
plt.xlabel("Circular Distance: Prev Estimate to Stimulus (deg)")
plt.ylabel("Angular Error (deg)")
plt.title("Prediction of Angular Error from Circular Distance")
plt.legend()
plt.axhline(0, color='gray', linestyle='--')
plt.grid(True)
plt.show()

### Binned Prediction Plot (Mean ± Std)

In [ ]:
import pandas as pd

# Bin the X values (e.g. into 20 bins)
df_plot = pd.DataFrame({
    "feature": X_test.flatten(),
    "true": y_test,
    "pred": y_pred
})

bins = np.linspace(-180, 180, 20)
df_plot["bin"] = pd.cut(df_plot["feature"], bins)

# Compute mean and std per bin
mean_true = df_plot.groupby("bin")["true"].mean()
mean_pred = df_plot.groupby("bin")["pred"].mean()
std_true = df_plot.groupby("bin")["true"].std()

# Plot
plt.figure(figsize=(8,5))
plt.errorbar(mean_true.index.categories.mid, mean_true, yerr=std_true, fmt='o', label="Mean True ± STD")
plt.plot(mean_pred.index.categories.mid, mean_pred, 'r--', label="Mean Predicted")
plt.xlabel("Binned Circular Distance (deg)")
plt.ylabel("Angular Error (deg)")
plt.title("Binned Fit of Angular Error vs Circular Distance")
plt.legend()
plt.grid(True)
plt.show()


## Hypothesis I Auxiliary Predictive Test: Effect of Absolute Circular Distance on Estimation Error

In our original model, we used the **signed circular distance** between the **previous trial’s estimate** and the **current trial’s stimulus direction** as the predictor. This assumes that the **direction** of the difference (clockwise vs counterclockwise) influences behavior.

However, we now hypothesize that **only the magnitude** of the difference matters — not the direction. That is, larger angular separations between past and current stimuli might lead to **grea**


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt

# --- Use absolute value of circular distance as the feature ---
df["abs_circ_dist"] = np.abs(df["circ_dist_prev_est_to_stim"])

# Drop NaNs
df_model = df.dropna(subset=["abs_circ_dist", "circular_error"])

# Prepare data
X = df_model[["abs_circ_dist"]].values
y = df_model["circular_error"].values

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit linear regression
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)

# Report R²
r2 = r2_score(y_test, y_pred)
print(f"R² score using absolute circular distance: {r2:.3f}")

# Plot using your specified format
plt.figure(figsize=(8,5))
plt.scatter(X_test, y_test, alpha=0.3, label="True")
plt.scatter(X_test, y_pred, color='red', alpha=0.3, label="Predicted")
plt.xlabel("Circular Distance: Prev Estimate to Stimulus (deg)")  # Keep original axis label
plt.ylabel("Angular Error (deg)")
plt.title("Prediction of Angular Error from Circular Distance")
plt.legend()
plt.axhline(0, color='gray', linestyle='--')
plt.grid(True)
plt.show()

## Hypothesis I Conclusion: Predictive Power of Circular Distance Features

We tested whether the **circular distance between the previous trial’s estimate and the current trial’s stimulus direction** could predict the **angular error** in human motion direction estimation.

Two models were compared:
- One using the **signed circular distance** as a feature.
- One using the **absolute circular distance** as a feature.
---
### Results
- The **signed circular distance** model yielded an R² ≈ 0.01
- The **absolute circular distance** model yielded an R² ≈ 0.000
---
### Interpretation
Both models explained **very little to no variance** in the estimation error, suggesting that:
- The circular distance between past and current trials alone is **not a strong linear predictor** of angular error.
- The relationship may be **nonlinear**, **subject-specific**, or influenced by **additional factors** such as:
  - Sensory uncertainty (e.g., motion coherence)
  - Prior distribution strength (`prior_std`)
  - Decision confidence or reaction time
  - Trial history beyond just the immediately previous trial
---
### Next Steps
To better understand human estimation behavior, future models should:
- Include **multiple features** in a multivariate regression
- Explore **nonlinear models**
- Consider subject-specific or hierarchical modeling
- Potentially use **Bayesian observer frameworks** that align with prior work


## Hypothesis II: Multivariate Predictors of Estimation Error

Based on previous results, we observed that the circular distance between the previous trial’s estimate and the current stimulus direction — whether signed or absolute — is **not a strong standalone predictor** of human angular error in this task.

We now hypothesize that:
> **A combination of multiple task-relevant features** can better predict the estimation error than any single feature alone.

### Features Included:
- `abs_circ_dist`: Absolute circular distance between previous estimate and current stimulus
- `motion_coherence`: Strength of sensory evidence (higher = less uncertainty)
- `prior_std`: Variability in the experimental prior (lower = stronger prior)
- `reaction_time`: May reflect decision confidence or task difficulty

We will fit a **multiple linear regression model** using these features and evaluate its performance using R² on a held-out test set.


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np

# --- Define features ---
features = ["abs_circ_dist", "motion_coherence", "prior_std", "reaction_time"]
df_model_multi = df.dropna(subset=features + ["circular_error"])

print(f"Sample size: {len(df_model_multi)} observations")
print(f"Missing data removed: {len(df) - len(df_model_multi)} observations")

# Prepare data
X = df_model_multi[features].values
y = df_model_multi["circular_error"].values

# Standardize features for better coefficient interpretation
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Fit model
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)

# Evaluation metrics
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
baseline_rmse = np.std(y_test)  # RMSE of always predicting mean

print(f"\nModel Performance:")
print(f"R² score: {r2:.3f}")
print(f"RMSE: {rmse:.3f}")
print(f"Baseline RMSE (predicting mean): {baseline_rmse:.3f}")
print(f"RMSE improvement: {((baseline_rmse - rmse) / baseline_rmse * 100):.1f}%")

# Statistical significance (rough approximation)
n = len(y_test)
if r2 > 0:
    f_stat = (r2 / (1 - r2)) * ((n - len(features) - 1) / len(features))
    print(f"Approximate F-statistic: {f_stat:.3f}")

# --- Enhanced Plot 1: Predicted vs. True values ---
plt.figure(figsize=(8, 6))
plt.scatter(y_test, y_pred, alpha=0.4, s=20)
plt.xlabel("True Angular Error (deg)")
plt.ylabel("Predicted Angular Error (deg)")
plt.title(f"Predicted vs. True Angular Error (R² = {r2:.3f})")

# Add perfect prediction line
min_val = min(y_test.min(), y_pred.min())
max_val = max(y_test.max(), y_pred.max())
plt.plot([min_val, max_val], [min_val, max_val], 'r--', alpha=0.8, label='Perfect Prediction')

# Add trend line
z = np.polyfit(y_test, y_pred, 1)
p = np.poly1d(z)
plt.plot(y_test, p(y_test), "b--", alpha=0.8, label=f'Actual Trend (slope={z[0]:.3f})')

plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

# --- Plot 2: Residuals vs. Predicted ---
residuals = y_test - y_pred

plt.figure(figsize=(8, 6))
plt.scatter(y_pred, residuals, alpha=0.4, s=20)
plt.axhline(0, color='red', linestyle='--', linewidth=2, label='Perfect Predictions')
plt.xlabel("Predicted Angular Error (deg)")
plt.ylabel("Residuals (True - Predicted)")
plt.title("Residuals vs. Predicted")
plt.legend()
plt.grid(True, alpha=0.3)

# Add LOESS smooth line to detect patterns
from scipy import stats
if len(y_pred) > 50:  # Only if enough points
    # Sort by predicted values for smooth line
    sort_idx = np.argsort(y_pred)
    y_pred_sorted = y_pred[sort_idx]
    residuals_sorted = residuals[sort_idx]

    # Simple moving average as LOESS approximation
    window = max(20, len(y_pred) // 20)
    smoothed = pd.Series(residuals_sorted).rolling(window=window, center=True).mean()
    plt.plot(y_pred_sorted, smoothed, 'orange', linewidth=2, label='Trend')
    plt.legend()

plt.show()

# --- Plot 3: Standardized Feature Coefficients ---
coef_df = pd.DataFrame({
    "Feature": features,
    "Coefficient": lr.coef_,
    "Abs_Coefficient": np.abs(lr.coef_)
}).sort_values("Abs_Coefficient", ascending=True)

plt.figure(figsize=(8, 6))
colors = ['red' if x < 0 else 'blue' for x in coef_df["Coefficient"]]
plt.barh(coef_df["Feature"], coef_df["Coefficient"], color=colors, alpha=0.7)
plt.xlabel("Standardized Coefficient")
plt.title("Feature Importance (Standardized Coefficients)")
plt.axvline(0, color='gray', linestyle='--')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# --- Additional Analysis: Feature Correlations ---
plt.figure(figsize=(8, 6))
feature_corr = df_model_multi[features + ["circular_error"]].corr()
sns.heatmap(feature_corr, annot=True, cmap='coolwarm', center=0,
            square=True, fmt='.3f')
plt.title("Feature Correlation Matrix")
plt.tight_layout()
plt.show()

# --- Feature Statistics ---
print(f"\nFeature Statistics:")
print(df_model_multi[features].describe())

print(f"\nFeature-Target Correlations:")
for feature in features:
    corr = df_model_multi[feature].corr(df_model_multi["circular_error"])
    print(f"{feature}: {corr:.3f}")

# --- Diagnostic: Check for outliers ---
plt.figure(figsize=(12, 3))
for i, feature in enumerate(features):
    plt.subplot(1, 4, i+1)
    plt.boxplot(df_model_multi[feature])
    plt.title(f"{feature}")
    plt.xticks([])
plt.tight_layout()
plt.suptitle("Feature Distributions (Check for Outliers)", y=1.02)
plt.show()

# --- Interpretation Summary ---
print(f"\n" + "="*50)
print("INTERPRETATION SUMMARY")
print("="*50)
print(f"• Model explains {r2*100:.1f}% of variance in angular error")
print(f"• RMSE is {rmse:.1f}°, vs baseline of {baseline_rmse:.1f}°")

if r2 < 0.05:
    print("• POOR MODEL FIT: Linear combination of these features")
    print("  does not effectively predict estimation error")
    print("• Possible reasons:")
    print("  - Non-linear relationships")
    print("  - Missing important predictors")
    print("  - High individual variability")
    print("  - Measurement noise dominates signal")

strongest_predictor = coef_df.loc[coef_df["Abs_Coefficient"].idxmax(), "Feature"]
print(f"• Strongest predictor: {strongest_predictor}")

print(f"\nRecommendations:")
print(f"• Consider non-linear models (polynomial, tree-based)")
print(f"• Include additional features (individual differences, trial history)")
print(f"• Check for interaction effects between features")
print(f"• Consider mixed-effects models to account for individual differences")

#  Hypothesis II Conclusion:

Model Performance

The extremely poor performance (R² = 0.001, RMSE = 50.182) indicates that this linear combination of features explains virtually none of the variance in angular estimation error. This suggests several important findings:

Linear relationships are insufficient: The features may have non-linear relationships with estimation error, or the relationships may be more complex than a simple additive model can capture.
Missing key predictors: The most important factors driving estimation error may not be included in your current feature set.
High individual variability: Human estimation error in this task may be dominated by factors not captured in these variables (e.g., attention, fatigue, individual differences in strategy).

Feature Effects (from the coefficient plot)

Motion coherence has the largest positive coefficient (around 3.5) suggesting higher coherence is associated with larger errors (counterintuitive)
Absolute circular distance has a moderate positive effect (around 0.8)
Reaction time has a smaller positive effect (around 0.7)
Prior standard deviation has minimal effect (near zero)

# Hypothesis III: Bayesian Linear Regression for Estimation Error Prediction

Our earlier linear models provided point estimates for feature weights but could not express uncertainty in those estimates. We now hypothesize that:

> A **Bayesian linear regression model** can quantify the **uncertainty in feature relationships**, especially for circular distance and other task-related variables.
---
### Why Bayesian Regression?
- It assumes a **prior distribution over the weights** (e.g., Gaussian prior on coefficients).
- It returns **posterior distributions**, not just point estimates.
- This allows us to measure **confidence** in whether each predictor contributes meaningfully to the prediction.
---
### Assumptions:
We assume:
- Angular error is normally distributed around a linear combination of features.
- Feature weights come from a zero-mean Gaussian prior.

We will use **Bayesian Ridge Regression** from `scikit-learn`, which approximates the Bayesian posterior and provides credible intervals for coefficients.


In [ ]:
from sklearn.linear_model import BayesianRidge
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np

# --- Features & target ---
features = ["abs_circ_dist", "motion_coherence", "prior_std", "reaction_time"]
df_model_bayes = df.dropna(subset=features + ["circular_error"])

X = df_model_bayes[features].values
y = df_model_bayes["circular_error"].values

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# --- Fit Bayesian Ridge Regression ---
bayes_model = BayesianRidge(compute_score=True)
bayes_model.fit(X_train, y_train)
y_pred = bayes_model.predict(X_test)

# --- Evaluation ---
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"R² (Bayesian Ridge): {r2:.3f}")
print(f"RMSE: {rmse:.3f}")

# --- Extract coefficient statistics ---
# Get posterior mean of coefficients
coef_mean = bayes_model.coef_

# Calculate posterior covariance and standard deviations
# For Bayesian Ridge, we can estimate the uncertainty using the posterior covariance
# Sigma = (alpha * I + beta * X.T @ X)^(-1) where alpha and beta are learned hyperparameters
X_train_centered = X_train - np.mean(X_train, axis=0)
precision_matrix = bayes_model.alpha_ * np.eye(X_train.shape[1]) + bayes_model.lambda_ * X_train_centered.T @ X_train_centered
covariance_matrix = np.linalg.inv(precision_matrix)
coef_std = np.sqrt(np.diag(covariance_matrix))

# Create DataFrame for coefficients
coef_df = pd.DataFrame({
    "Feature": features,
    "Mean": coef_mean,
    "StdDev": coef_std
})

print("\nCoefficient Statistics:")
print(coef_df)

# --- Plot 1: Predictions vs. True values ---
plt.figure(figsize=(7,5))
plt.scatter(y_test, y_pred, alpha=0.3)
plt.xlabel("True Angular Error (deg)")
plt.ylabel("Predicted Angular Error (deg)")
plt.title("Bayesian Ridge: Predicted vs. True Angular Error")
plt.axhline(0, color='gray', linestyle='--')
plt.axvline(0, color='gray', linestyle='--')
# Add diagonal line for perfect predictions
min_val = min(y_test.min(), y_pred.min())
max_val = max(y_test.max(), y_pred.max())
plt.plot([min_val, max_val], [min_val, max_val], 'r--', alpha=0.8, label='Perfect Prediction')
plt.legend()
plt.grid(True)
plt.show()

# --- Plot 2: Residuals vs. Predicted ---
residuals = y_test - y_pred
plt.figure(figsize=(7,5))
plt.scatter(y_pred, residuals, alpha=0.3)
plt.axhline(0, color='red', linestyle='--')
plt.xlabel("Predicted Angular Error (deg)")
plt.ylabel("Residuals (True - Predicted)")
plt.title("Residuals vs. Predicted (Bayesian Ridge)")
plt.grid(True)
plt.show()

# --- Plot 3: Coefficients with uncertainty using matplotlib ---
plt.figure(figsize=(8,5))

# Horizontal bar plot with error bars
plt.barh(
    y=coef_df["Feature"],
    width=coef_df["Mean"],
    xerr=coef_df["StdDev"],
    align="center",
    alpha=0.7,
    color="skyblue",
    ecolor="black",
    capsize=5
)

plt.xlabel("Posterior Mean Coefficient")
plt.title("Posterior Mean and Uncertainty (Bayesian Ridge)")
plt.axvline(0, color='gray', linestyle='--')
plt.grid(True)
plt.tight_layout()
plt.show()

# --- Additional: Plot coefficient credible intervals ---
plt.figure(figsize=(8,6))

# Calculate 95% credible intervals (assuming normal posterior)
ci_lower = coef_df["Mean"] - 1.96 * coef_df["StdDev"]
ci_upper = coef_df["Mean"] + 1.96 * coef_df["StdDev"]

# Create error bar plot
plt.errorbar(
    x=coef_df["Mean"],
    y=range(len(features)),
    xerr=1.96 * coef_df["StdDev"],
    fmt='o',
    capsize=5,
    capthick=2,
    markersize=8
)

plt.yticks(range(len(features)), features)
plt.xlabel("Coefficient Value")
plt.ylabel("Features")
plt.title("Bayesian Ridge Coefficients with 95% Credible Intervals")
plt.axvline(0, color='red', linestyle='--', alpha=0.7)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# --- Print model hyperparameters ---
print(f"\nLearned Hyperparameters:")
print(f"Alpha (precision of noise): {bayes_model.alpha_:.6f}")
print(f"Lambda (precision of coefficients): {bayes_model.lambda_:.6f}")
print(f"Log marginal likelihood: {bayes_model.scores_[-1]:.3f}")

# Hypothesis III Conclusion: Bayesian Linear Regression Interpretation

- **R² score**: 0.001  
- **RMSE**: ~50.19 degrees

These results indicate that the model explained **virtually no variance** in the angular error. The RMSE confirms that the predictions are highly inaccurate on average.

---
#### 1. Predicted vs. True Angular Error:
- The predicted values are tightly clustered around 0.
- True values span the full range of angular error.
- The model is clearly **underfitting** and fails to capture the signal in the data.
---
#### 2. Posterior Coefficient Estimates:
- Most coefficients are close to zero with **large uncertainty bounds**.
- `prior_std` shows a slightly positive coefficient, suggesting a **minor effect** on error, consistent with prior knowledge.
- Overall, the model indicates **low confidence** in any single feature being predictive.
---
### Conclusion:

- Angular error is **not linearly predictable** using these features.
- The Bayesian framework gives useful uncertainty estimates, which reveal weak or unreliable feature effects.


### Export requirements file

In [ ]:
!pip freeze > requirements.txt


In [ ]:
from google.colab import files
files.download("requirements.txt")

### Fix metadata

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!pip install nbstripout
!nbstripout "Regression_Models.ipynb"

In [ ]:
import json, io

in_path  = "Regression_Models.ipynb"
out_path = "Regression_Models.ipynb"

with io.open(in_path, "r", encoding="utf-8") as f:
    nb = json.load(f)

# Remove top-level widgets metadata if present
nb.get("metadata", {}).pop("widgets", None)

# Remove any cell-level widgets metadata if present
for cell in nb.get("cells", []):
    cell.get("metadata", {}).pop("widgets", None)

with io.open(out_path, "w", encoding="utf-8") as f:
    json.dump(nb, f, ensure_ascii=False, indent=1)


In [ ]:
files.download("Regression_Models.ipynb")

## Acknowledgements:
Daphne Zhang, Jacob Boulrice, Shayla Schwartz, Yi Gao, Claude Sonnet 4, GPT-4o